In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
from datetime import datetime


In [34]:
data=pd.read_csv('../data/raw_analyst_ratings.csv')
data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [35]:
data.shape


(1407328, 6)

In [36]:
data.info

<bound method DataFrame.info of          Unnamed: 0                                           headline  \
0                 0            Stocks That Hit 52-Week Highs On Friday   
1                 1         Stocks That Hit 52-Week Highs On Wednesday   
2                 2                      71 Biggest Movers From Friday   
3                 3       46 Stocks Moving In Friday's Mid-Day Session   
4                 4  B of A Securities Maintains Neutral on Agilent...   
...             ...                                                ...   
1407323     1413844             Top Narrow Based Indexes For August 29   
1407324     1413845  Recap: Wednesday's Top Percentage Gainers and ...   
1407325     1413846  UPDATE: Oppenheimer Color on China Zenix Auto ...   
1407326     1413847  Oppenheimer Initiates China Zenix At Outperfor...   
1407327     1413848  China Zenix Auto International Opens For Tradi...   

                                                       url          publisher  

In [37]:
# Calculate the length of each headline
data['headline_length'] = data['headline'].apply(len)

# Obtain basic statistics for headline lengths
headline_stats = data['headline_length'].describe()
print(headline_stats)

count    1.407328e+06
mean     7.312051e+01
std      4.073531e+01
min      3.000000e+00
25%      4.700000e+01
50%      6.400000e+01
75%      8.700000e+01
max      5.120000e+02
Name: headline_length, dtype: float64


In [38]:
# Count the number of articles per publisher
publisher_counts = data['publisher'].value_counts()
print(publisher_counts)


publisher
Paul Quintaro        228373
Lisa Levin           186979
Benzinga Newsdesk    150484
Charles Gross         96732
Monica Gerson         82380
                      ...  
MoneyGeek                 1
muathe                    1
Robert Morris             1
LeftCoastHedgie           1
Jeremie Capron            1
Name: count, Length: 1034, dtype: int64


In [43]:
# Convert the 'date' column to datetime, allowing pandas to infer the format
# errors='coerce' will convert any non-convertible values to NaT (Not a Time) rather than raising an error.

data['date'] = pd.to_datetime(data['date'], errors='coerce')

# Count the number of articles per day
daily_counts = data['date'].dt.date.value_counts().sort_index()
print(daily_counts)


date
2011-04-27      1
2011-04-28      2
2011-04-29      2
2011-04-30      1
2011-05-01      1
             ... 
2020-06-07     25
2020-06-08    765
2020-06-09    804
2020-06-10    806
2020-06-11    544
Name: count, Length: 2528, dtype: int64


In [45]:
# Function to get the sentiment polarity
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity



In [47]:
# Apply the sentiment analysis
data['sentiment'] = data['headline'].apply(get_sentiment)



In [48]:
# Classify the sentiment as positive, negative, or neutral
data['sentiment_label'] = data['sentiment'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))

# Display sentiment analysis results
print(data[['headline', 'sentiment', 'sentiment_label']])

                                                  headline  sentiment  \
0                  Stocks That Hit 52-Week Highs On Friday       0.00   
1               Stocks That Hit 52-Week Highs On Wednesday       0.00   
2                            71 Biggest Movers From Friday       0.00   
3             46 Stocks Moving In Friday's Mid-Day Session       0.00   
4        B of A Securities Maintains Neutral on Agilent...       0.00   
...                                                    ...        ...   
1407323             Top Narrow Based Indexes For August 29       0.15   
1407324  Recap: Wednesday's Top Percentage Gainers and ...       0.15   
1407325  UPDATE: Oppenheimer Color on China Zenix Auto ...       0.00   
1407326  Oppenheimer Initiates China Zenix At Outperfor...       0.00   
1407327  China Zenix Auto International Opens For Tradi...       0.00   

        sentiment_label  
0               neutral  
1               neutral  
2               neutral  
3               neu

In [56]:
import gensim


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject